In [9]:
import pandas as pd
import xgboost as xgb
import os
import logging
import time
import cPickle as pickle
import numpy as np

# from IPython.utils.io import Tee
# wtf = Tee('../logs/%s.log' % str(int(time.time())), 'w', channel='stderr')

In [ ]:
%%time
PATH_STORE = '../saved/storage.h5'
train_pd = pd.read_hdf(PATH_STORE, 'train')

In [ ]:
%%time
# Out-of-memory
PATH_LIBSVM = '/tmp/train.libsvm'
PATH_CACHE = '/tmp/dtrain.cache'
if os.path.exists(PATH_CACHE):
    print 'Remove old cache @ %s' % PATH_CACHE
    os.remove(PATH_CACHE)
    os.remove(PATH_CACHE + '.row.blob')

# dtrain = xgb.DMatrix('../saved/med.libsvm#%s' % PATH_CACHE)
dtrain = xgb.DMatrix('%s#%s' % (PATH_LIBSVM, PATH_CACHE))

In [2]:
%%time
# In-memory
PATH_LIBSVM = '/tmp/train.libsvm'
dtrain = xgb.DMatrix(PATH_LIBSVM)

CPU times: user 40.1 s, sys: 1.56 s, total: 41.6 s
Wall time: 20.1 s


In [10]:
params = {}
params['objective'] = "binary:logistic"
params['eta'] = 0.01
params['min_child_weight'] = 6
params['subsample'] = 0.7
params['colsample_bytree'] = 0.5
# params['max_delta_step'] = 1
params['silent'] = 1
params['max_depth'] = 9 #7
params['eval_metric'] = 'auc'
# params["scale_pos_weight"] = 1.0
# params['gamma'] = 1
params['alpha'] = 1
# params['nthread'] = 4

def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label==1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

plst = list(params.items())

max_rounds = 4000


In [ ]:
%%time
cv_log = xgb.cv(params, dtrain, max_rounds, nfold=3,
                metrics={'auc'}, seed = 0,
                fpreproc=fpreproc)

pickle.dump(cv_log, open('../saved/cv_log_%d.p' % int(time.time()), 'wb'))

In [5]:
y = dtrain.get_label()

In [6]:
from collections import Counter
Counter(y)

Counter({0.0: 111415, 1.0: 33760})

In [12]:
cv_log[-1000:]

['[3000]\tcv-test-auc:0.791934+0.001420\tcv-train-auc:0.999252+0.000049',
 '[3001]\tcv-test-auc:0.791935+0.001420\tcv-train-auc:0.999253+0.000049',
 '[3002]\tcv-test-auc:0.791937+0.001418\tcv-train-auc:0.999255+0.000050',
 '[3003]\tcv-test-auc:0.791939+0.001417\tcv-train-auc:0.999257+0.000049',
 '[3004]\tcv-test-auc:0.791940+0.001419\tcv-train-auc:0.999260+0.000048',
 '[3005]\tcv-test-auc:0.791939+0.001418\tcv-train-auc:0.999262+0.000048',
 '[3006]\tcv-test-auc:0.791941+0.001419\tcv-train-auc:0.999263+0.000049',
 '[3007]\tcv-test-auc:0.791943+0.001418\tcv-train-auc:0.999266+0.000049',
 '[3008]\tcv-test-auc:0.791942+0.001412\tcv-train-auc:0.999268+0.000049',
 '[3009]\tcv-test-auc:0.791940+0.001414\tcv-train-auc:0.999270+0.000049',
 '[3010]\tcv-test-auc:0.791941+0.001414\tcv-train-auc:0.999271+0.000048',
 '[3011]\tcv-test-auc:0.791939+0.001413\tcv-train-auc:0.999273+0.000049',
 '[3012]\tcv-test-auc:0.791940+0.001416\tcv-train-auc:0.999274+0.000049',
 '[3013]\tcv-test-auc:0.791943+0.00141

In [3]:
int(time.time())

1441215081